In [ ]:
# Solution Ex. 44 - Version based on left outer join

In [ ]:
inputPathWatched = "/data/students/bigdata-01QYD/ex_data/Ex44/data/watchedmovies.txt"
inputPathPreferences = "/data/students/bigdata-01QYD/ex_data/Ex44/data/preferences.txt"
inputPathMovies = "/data/students/bigdata-01QYD/ex_data/Ex44/data/movies.txt"
outputPath = "res_out_Ex44/"
threshold = 0.5

In [ ]:
# Read the content of the watched movies file
watchedRDD = sc.textFile(inputPathWatched)

In [ ]:
# Select only userid and movieid
# Define an RDD or pairs with movieid as key and userid as value
movieUserPairRDD = watchedRDD\
.map(lambda line:  (line.split(",")[1], line.split(",")[0]))

In [ ]:
# Read the content of the movies file
moviesRDD = sc.textFile(inputPathMovies)

In [ ]:
# Select only movieid and genre
# Define an RDD of pairs with movieid as key and genre as value
movieGenrePairRDD = moviesRDD\
.map(lambda line:  (line.split(",")[0], line.split(",")[2]))

In [ ]:
# Join watched movies with movies
joinWatchedGenreRDD = movieUserPairRDD.join(movieGenrePairRDD)

In [ ]:
#joinWatchedGenreRDD.collect()

In [ ]:
# Select only userid and movie genre
# Return pairs(userid+movie genre, None)
usersWatchedGenresRDD = joinWatchedGenreRDD.map(lambda p: ( (p[1][0], p[1][1]), None) )

In [ ]:
#usersWatchedGenresRDD.collect()

In [ ]:
# Read the content of preferences.txt
preferencesRDD = sc.textFile(inputPathPreferences)

In [ ]:
# Define an RDD of pairs with userid+movie genere as key and a constant, not queal to None, as value 
# (I decided to use "LikedMovieGenre" for "explainability")
userLikedGenresRDD = preferencesRDD\
.map(lambda line: ( (line.split(",")[0], line.split(",")[1]), "LikedMovieGenre")  )

In [ ]:
#userLikedGenresRDD.collect()

In [ ]:
# Apply left outer join to find both matchings and mismatchings between 
# "User watched a movie genre" and "User likes/does not like a movie genre"
userWatchedLikedNotLikedGenres = usersWatchedGenresRDD\
.leftOuterJoin(userLikedGenresRDD)

In [ ]:
#userWatchedLikedNotLikedGenres.collect()

In [ ]:
# Map each pair to a new pair with
# - Key = userId
# - value = (1, 0/1). The second part is 1 is the used watched a movie genre 
#                     he/she does not like. Otherwise, 0.


def mapLikedNotLiked(pair):
    userId=pair[0][0]
    likedMovieGenre=pair[1][1]
    
    # If likedMovieGenre is None the user watched a movie genre he/she does not like
    if (likedMovieGenre is None):
        return (userId, (1,1)) # One more visualization - One more mismatch
    else:
        return (userId, (1,0)) # One more visualization - One more matching
    

userWatchedNotLiked = userWatchedLikedNotLikedGenres.map(mapLikedNotLiked)

In [ ]:
#userWatchedNotLiked.collect()

In [ ]:
# Count for each user the number of "visualizations/watched movies" and the number 
# of unliked "visualizations/watched movies"
userCountWatchedAndNotLiked = userWatchedNotLiked\
.reduceByKey(lambda v1, v2: (v1[0]+v2[0],v1[1]+v2[1]) )

In [ ]:
#userCountWatchedAndNotLiked.collect()

In [ ]:
# Compute the "misleading index"
userMisleadingIndex = userCountWatchedAndNotLiked.mapValues(lambda v: float(v[1])/float(v[0]))

In [ ]:
#userMisleadingIndex.collect()

In [ ]:
# Filter/select the users with a misleading profile
misleadingUsersListsRDD = userMisleadingIndex\
.filter(lambda p: p[1]>threshold)

In [ ]:
# Select only the userid of the users with a misleading profile
misleadingUsersRDD = misleadingUsersListsRDD.keys()

In [ ]:
#misleadingUsersRDD.collect()

In [ ]:
misleadingUsersRDD.saveAsTextFile(outputPath)